In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_community.adapters.openai import convert_openai_messages

In [2]:
os.environ["LANGSMITH_TRACING"] = "true"

load_dotenv("../.env")

True

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

In [5]:
@tool
def get_ticket_price(destination_city):
    """
    Get the price of a return ticket to the destination city.
    Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city?'.
    """
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

@tool
def book_flight_to(destination_city):
    """
    Book a flight to the destination city.
    Call this whenever you need to book a flight, for example when a customer asks 'Can you book a flight to this city for me?'.
    """
    if destination_city.lower() in ticket_prices.keys():
        with open("bookings.txt", "a") as f:
            f.write(f"Flight booked to {destination_city}\n")
    else:
        return "We don't fly to that city. Please choose from London, Paris, Tokyo, or Berlin."
    return f"Flight to {destination_city} has been booked"

In [6]:
get_ticket_price.invoke("London")

'$799'

In [7]:
model = init_chat_model("gpt-4o-mini", model_provider="openai")
tools = [get_ticket_price, book_flight_to]
model_with_tools = model.bind_tools(tools)

In [8]:
def chat(message, history):
    """
    Chat with the model, using the tools to get ticket prices and book flights.
    """
    messages = [SystemMessage(content=system_message)] + convert_openai_messages(history) + [HumanMessage(content=message)]
    model_with_tools = model.bind_tools(tools)
    ai_msg = model_with_tools.invoke(messages)
    messages.append(ai_msg)
    
    for tool_call in ai_msg.tool_calls:
        selected_tool = {"get_ticket_price": get_ticket_price, "book_flight_to": book_flight_to}[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)
    
    response = model_with_tools.invoke(messages)
    return response.content

In [9]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
